In [16]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm
import re

from urllib.parse import urljoin

In [26]:
def each_page_articles_links(url, base_url):
    """
    
    """

    
    final_response = {
        "Links": [],
        "Message": "Success",
        "Response": 200,
        "source_url": url
    }
    try:
        headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-encoding': 'gzip, deflate, br, zstd',
            'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
            'cache-control': 'max-age=0',
            'cookie': 'VEE=wikitext; bowikimwclientpreferences=vector-feature-limited-width-clientpref-0; GeoIP=IN:::22.00:79.00:v4; bowikimwuser-sessionId=57993a543a8ba6c65a70; WMF-Last-Access=25-Sep-2024; WMF-Last-Access-Global=25-Sep-2024; NetworkProbeLimit=0.001; bowikiPreventProbe=1',
            'priority': 'u=0, i',
            'referer': 'https://bo.wikipedia.org/wiki/%E0%BD%82%E0%BD%99%E0%BD%BC%E0%BC%8B%E0%BD%84%E0%BD%BC%E0%BD%A6%E0%BC%8D',
            'sec-ch-ua': '"Microsoft Edge";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'
        }
        start_time = time.time()
        response = requests.get(url, headers=headers, timeout=(5, 60-5))
        response.raise_for_status()
        end_time = time.time()

        if end_time-start_time > 50:
            print(f"This ULR Took more then 50s: {url}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # # Getting all the links of articles 
        all_links = []
        next_page_link = "No Next Page"
        Next_page_name = ""
        all_link_article = soup.find("div", class_="mw-body-content")
        if all_link_article:
            next_page_content = all_link_article.find("div", class_="mw-allpages-nav")
            if next_page_content:
                next_page_link_box = next_page_content.find_all("a")
                if next_page_link_box:
                    next_page_link = urljoin(base_url, next_page_link_box[-1].get("href"))

            page_content = all_link_article.find("div", class_="mw-allpages-body")
            if page_content:
                all_content_link = page_content.find_all("li")
                if all_content_link:
                    for each_link in all_content_link:
                        link_box = each_link.find("a")
                        if link_box:
                            link = link_box.get("href")
                            url = urljoin(base_url, link)
                            all_links.append(url)
                    
        final_response["Links"] = all_links
        return final_response, next_page_link
     
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response, None
    except requests.RequestException as e:
        # print(f"An error occurred while fetching the webpage: {e}")
        final_response["Message"] = f"An error occurred while fetching the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', None)
        return final_response, None
    except ValueError as e:
        # print(f"An error occurred while parsing the webpage: {e}")
        final_response["Message"] = f"An error occurred while parsing the webpage: {e}"
        final_response["Response"] = 404
        # getattr(e.response, 'status_code', None)
        return final_response, None
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response, None

In [29]:
url = "https://bo.wikipedia.org/wiki/Special:AllPages"
base_url = "https://bo.wikipedia.org"
file_data, next_page_link = each_page_articles_links(url, base_url)
file_data, next_page_link = each_page_articles_links(next_page_link, base_url)
next_page_link

'https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BD%80%E0%BC%8B%E0%BD%96%E0%BC%8B%E0%BD%A3%E0%BD%B2'

In [30]:
def save_json(path, file_name, data):
    """
    
    """
    with open(path+file_name, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)
        print(f"Successfully saved: {file_name}")


In [32]:
%%time


Page_index = 1
tril_count = 0
path = "./data/links/"


url = "https://bo.wikipedia.org/wiki/Special:AllPages"
base_url = "https://bo.wikipedia.org"
while 1:
    
    print(url)
    file_data, next_page_link = each_page_articles_links(url, base_url)
    if next_page_link:
        if next_page_link == "No Next Page":
            print("WE REACH TO THE END")
            print(f"Final Page No: {Page_index}")
            break
        url = next_page_link
        file_name = f"Page {Page_index}.json" 
        save_json(path, file_name, file_data)
        Page_index += 1
    else:
        tril_count += 1
        print(f"failed in {url}: tril_count: {tril_count}")

    if tril_count>3:
        break

https://bo.wikipedia.org/wiki/Special:AllPages
Successfully saved: Page 1.json
https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BC%A1%E0%BC%A9%E0%BC%A1%E0%BC%A8%E0%BD%A3%E0%BD%BC%E0%BC%8D
Successfully saved: Page 2.json
https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BD%80%E0%BC%8B%E0%BD%96%E0%BC%8B%E0%BD%A3%E0%BD%B2
Successfully saved: Page 3.json
https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BD%80%E0%BD%B4%E0%BD%93%E0%BC%8B%E0%BD%96%E0%BD%8F%E0%BD%B4%E0%BD%A6%E0%BC%8B%E0%BD%A2%E0%BD%B2%E0%BD%82%E0%BC%8B%E0%BD%98%E0%BD%9B%E0%BD%BC%E0%BD%91%E0%BC%8B
Successfully saved: Page 4.json
https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BD%80%E0%BE%B2%E0%BD%B2%E0%BC%8B%E0%BD%85%E0%BD%84%E0%BC%8B
Successfully saved: Page 5.json
https://bo.wikipedia.org/w/index.php?title=Special:AllPages&from=%E0%BD%81%E0%BD%82%E0%BC%8B%E0%BD%81%E0%BD%B4%E0%BD%A2%E0%BC%8B%E0%BD%A3%E0%BC%8B%E0%BD%82%E0%BD%85%E0%BD%BA%E0%BD%A6%E0%B

KeyboardInterrupt: 